In [1]:
# Import
from dotenv import load_dotenv
import os
import sys
import requests
from requests import post
import json
import time
import pandas as pd

In [2]:
load_dotenv()

client_id_igdb = os.getenv("IGDB_ID_CLIENT")
client_secret_igdb = os.getenv("IGDB_SECRET_CLIENT")

In [3]:
print(client_id_igdb)
print(client_secret_igdb)

wxbwlcskmokd9wdad9b24663j01q97
80utj76in1slffnnrruzm67wvc7il8


In [4]:
# Récupération du token d'accès pour IGDB

url = f"https://id.twitch.tv/oauth2/token?client_id={client_id_igdb}&client_secret={client_secret_igdb}&grant_type=client_credentials"

print(url)
response = requests.post(url)
if response.status_code == 200:
    response_json = response.json()
    if 'expires_in' in response_json == 0:
        print(f"Le token a expiré, il faut le renouveler")
    access_token = response_json['access_token']
    print(f"Access Token: {access_token}")
else:
    print("Error fetching data from the API")
    print("Status code:", response.status_code)

https://id.twitch.tv/oauth2/token?client_id=wxbwlcskmokd9wdad9b24663j01q97&client_secret=80utj76in1slffnnrruzm67wvc7il8&grant_type=client_credentials
Access Token: ooyghy5l7u9pv3zx90cx98bxgus8e7


In [5]:
# Récuperer le nombres de jeux
url_igdb_engines_count = "https://api.igdb.com/v4/games/count"

headers = {
    'Client-ID': client_id_igdb,   
    'Authorization': f'Bearer {access_token}',
    }

response = post(url_igdb_engines_count, headers=headers, data='fields *;')
print(response.json())

{'count': 320869}


In [15]:
# Récupération des games


headers_game = {
    'Client-ID': client_id_igdb,
    'Authorization': f'Bearer {access_token}',
}

response = post('https://api.igdb.com/v4/games', headers=headers_game , data='fields name,age_ratings,category,dlcs,first_release_date,game_engines,game_status,genres,language_supports,multiplayer_modes,videos,total_rating,total_rating,summary,player_perspectives, platforms,slug  ; limit 500;')
# Vérification de la réponse
if response.status_code == 200:
    # On transforme la réponse en JSON
    response_json = response.json()
    # On transforme le JSON en DataFrame
    df_games = pd.DataFrame(response_json)
else:
    print("Error fetching data from the API")
    print("Status code:", response.status_code)
    
df_games

,id,category,game_engines,genres,name,platforms,player_perspectives,slug,summary,videos,first_release_date,age_ratings,language_supports,total_rating,multiplayer_modes,game_status,dlcs
0,330684,14,[439],"[10, 33]",Nightmare Kart: The Old Karts,[6],[2],nightmare-kart-the-old-karts,An upcoming free expansion to Nightmare Kart w...,[139717],NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,177310,0,[750],"[13, 32]",The Undying Beast,[6],[1],the-undying-beast,"There was a flash of light, a choir of glass, ...",NaN,1.586736e+09,NaN,NaN,NaN,NaN,NaN,NaN
2,43460,3,NaN,[25],Sengoku Anthology,[8],NaN,sengoku-anthology,Sengoku Anthology is a collection of the three...,NaN,1.251763e+09,[15301],NaN,NaN,NaN,NaN,NaN
3,282079,5,NaN,"[5, 8]",Rockman X: New Year 2023,[58],"[2, 4]",rockman-x-new-year-2023,A small hack to celebrate the Chinese New Year...,NaN,1.672531e+09,NaN,[876934],NaN,NaN,NaN,NaN
4,63844,0,NaN,[14],Ace wo Nerae!,[58],[2],ace-wo-nerae,A tennis game for the Super Famicom based on t...,[43074],7.565184e+08,NaN,[685891],52.904629,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,63640,0,[64],[4],Super Brawl,[82],[4],super-brawl,The initial release of Nickelodeon's Super Bra...,"[102237, 102238]",1.293754e+09,NaN,[852524],NaN,NaN,NaN,NaN
496,257872,0,[64],"[10, 14]",Renegade Racing,[82],[4],renegade-racing,Complete various missions as you race through ...,"[137939, 137940]",1.360195e+09,NaN,[1149877],NaN,NaN,NaN,NaN
497,339629,0,NaN,[31],I Mother,NaN,NaN,i-mother,Embark on an adventure as a solitary Neanderth...,[145696],NaN,NaN,"[1211790, 1211791, 1211792]",NaN,NaN,NaN,NaN
498,289145,0,[64],[8],Snow Drift,[82],[4],snow-drift,Take out penguins and avoid the arctic enemies...,[108348],1.197590e+09,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
def get_all_games():
    all_games = []
    offset = 0
    limit = 500  # Maximum limit for IGDB API

    while True:
        response = post('https://api.igdb.com/v4/games', headers=headers_game, data=f'fields name,age_ratings,category,dlcs,first_release_date,game_engines,game_status,genres,language_supports,multiplayer_modes,videos,total_rating,total_rating,summary,player_perspectives,slug, platforms ; limit {limit}; offset {offset};')
        
        if response.status_code == 200:
            response_json = response.json()
            if not response_json:  # If no more data, break the loop
                break
            all_games.extend(response_json)
            offset += limit
        else:
            print("Error fetching data from the API")
            print("Status code:", response.status_code)
            break

    return pd.DataFrame(all_games)

In [ ]:
df_games = get_all_games()

In [19]:
# Transformer la colonne first_release_date en datetime
df_games['first_release_date'] = pd.to_datetime(df_games['first_release_date'], unit='s')

In [20]:
df_games.to_csv('../csv/games.csv', index=False)